In [1]:
import melign

In [2]:
dataset = melign.Dataset("dataset_test")
assert dataset.is_full()
print(f"Length of dataset: {len(dataset)}")

Length of dataset: 186


In [4]:
result_df, alignments = melign.eval_pipeline(
    melign.get_melodics_config(hop_length=1),
    dataset,
    verbose=1,
    baseline=False,
    save_plot=True,
    save_mid=True,
    n_jobs=12)

Processing songs (n_jobs=12):   0%|          | 0/186 [00:00<?, ?it/s]

In [4]:
result_df

,Sample,Alignment_Precision,Alignment_Recall,Alignment_F1,Runtime_seconds
0,Letting Go(蔡健雅),0.948925,0.710262,0.812428,3.639424
1,Mean,0.948925,0.710262,0.812428,3.639424
2,Median,0.948925,0.710262,0.812428,3.639424


In [4]:
melign.align.predict_f1(alignments["Letting Go(蔡健雅)"])

0.9125597457706844

In [5]:
from typing import TypedDict
from melign.align.alignment import Alignment

class SelfEvalResult(TypedDict):
    pred_f1: float
    above_between: float
    between: float
    error: float
    miss: float
    shadow: float

def self_eval(alignment: Alignment) -> SelfEvalResult:
    above_between = sum(match.sum_above_between for match in alignment.matches) / len(alignment.events)
    between = sum(match.sum_between for match in alignment.matches) / len(alignment.events)
    error = sum(match.sum_error for match in alignment.matches) / len(alignment.events)
    miss = sum(match.sum_miss for match in alignment.matches) / len(alignment.events)
    shadow = sum(match.sum_shadow for match in alignment.matches) / len(alignment.events)
    pred = 0.9950128655273125 \
         - 0.6607903283221324 * between \
         - 0.5518402627709131 * miss \
         - 0.32216793174475333 * error \
         - 0.74385038669203 * shadow
    return SelfEvalResult(
        pred_f1=pred,
        above_between=above_between,
        between=between,
        error=error,
        miss=miss,
        shadow=shadow
    )


In [8]:
result_df

,Sample,Alignment_Precision,Alignment_Recall,Alignment_F1,Runtime_seconds
0,Letting Go(蔡健雅),0.930521,0.735294,0.821468,63.533850
1,On My Way(萨吉),0.951567,0.876640,0.912568,41.148620
2,Speechless(Naomi Scott),0.987212,0.957816,0.972292,36.973145
3,The Best For You(欧阳娜娜),0.971861,0.935417,0.953291,37.971167
4,Too Bad(林俊杰),0.868085,0.969121,0.915825,24.725317
...,...,...,...,...,...
183,骗(张碧晨),0.979466,0.920849,0.949254,51.533294
184,黑夜问白天(林俊杰),0.990632,0.948430,0.969072,34.921188
185,黑暗骑士(林俊杰、五月天),0.984544,0.960784,0.972519,60.426982
186,Mean,0.958520,0.941610,0.948992,44.415079


In [14]:
self_eval(alignments["黑暗骑士(林俊杰、五月天)"])

{'pred_f1': 0.9546192374434291,
 'above_between': 0.0,
 'between': 0.03554868624420402,
 'error': 0.007461118624418168,
 'miss': 0.02627511591962906,
 'shadow': 0.0}